## mimlsvm

mi = mulit-instance
ml = multi-label
svm = svm

As described in Shen et al 2009: http://ieeexplore.ieee.org/document/5346261/

> Should we use SVM-struct instead? https://github.com/pystruct/pystruct


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
import nltk
import os
import re
import pickle
import sklearn
import sys
import string

from hausdorff import hausdorff

from nltk import TextTilingTokenizer
from scipy.spatial.distance  import directed_hausdorff, pdist

from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score, GridSearchCV,ParameterGrid, train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler,MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer
from sklearn.svm import SVC

from tqdm import *

%matplotlib inline
%load_ext autoreload
%autoreload 1

In [2]:
src_dir = os.path.join(os.getcwd(), os.pardir, '../src')
sys.path.append(src_dir)

In [3]:
%aimport data.movielens_20m_imdb
%aimport helpers.labels,helpers.neighbours, helpers.segments
%aimport utils.dataframes, utils.clusters

In [4]:
from data.movielens_20m_imdb import load_or_get_from_cache
from helpers.labels import truncate_labels
from helpers.neighbours import get_predicted_labels_from_neighbours
from helpers.segments import make_distance_matrix_for_segments,vectorize_segments

from utils.dataframes import sample_rows
from utils.clusters import k_medoids

In [5]:
INTERIM_DATA_ROOT = os.path.abspath("../../data/interim/movielens-ml20m-imdb/")
ML_ROOT = "/media/felipe/SAMSUNG/movielens/ml-20m/"
IMDB_ROOT = "/media/felipe/SAMSUNG/imdb/"

PATH_TO_MOVIES = ML_ROOT + "/movies.csv"
PATH_TO_TAG_ASSIGNMENTS = ML_ROOT + "/tags.csv"
PATH_TO_MOVIE_PLOTS = IMDB_ROOT+"/plot.list"

# CONFIGS

# preprocessing

MAX_NB_WORDS = 300
PREPROC=None
STOP_WORDS=[None,'english']
VECTORIZER_NORM = ['l1','l2',None]

# for sampling
NB_DOCS = 500

W=20 # Pseudosentence size (in words) - not specified in the paper, taken from TextTiling default values
K=10 # Size (in sentences) of the block used in the block comparison method - not specified in the paper, taken from TextTiling default values

MIN_LABEL_DF = 5 # like in the paper

# calculating medoids
MEDOID_NORMALIZATION = ['standard','minmax',None] # not specified in the paper
SAMPLE_TO_NB_MEDOIDS_RATIO = 0.2 # not specified in the paper, but taken from MIMLSVM canonical implementation

# classification
SVM_KERNEL=['poly'] # not specified in the paper, but taken from MIMLSVM canonical implementation
SVM_GAMMA=[0.1,0.2,0.5,1.0] # not specified in the paper, but taken from MIMLSVM canonical implementation
SVM_C= [0.01,0.1,1]# not specified in the paper, but taken from MIMLSVM canonical implementation
SVM_DEGREE=[3,4,5]

In [6]:
docs_df = load_or_get_from_cache(PATH_TO_MOVIES,PATH_TO_TAG_ASSIGNMENTS,PATH_TO_MOVIE_PLOTS,INTERIM_DATA_ROOT)

In [7]:
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [8]:
# remove this for production
docs_df = sample_rows(docs_df,NB_DOCS)

In [9]:
docs_df['sentences'] = docs_df['plot'].map(lambda row: sentence_tokenizer.tokenize(row))

In [10]:
%%time
docs_df['num_sentences'] = docs_df['sentences'].map( lambda sents: len(sents))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1 ms


In [11]:
docs_df.head()

,movie_id,title,unique_tags,num_users,num_unique_tags,plot,sentences,num_sentences
0,4153,Down to Earth (2001),"ghostsafterlife,remakeofhere-comes-mr-jordan,c...",5.0,9,It seems everyone is trying to get into heaven...,[It seems everyone is trying to get into heave...,15
1,31038,Smooth Talk (1985),"sundance-award-winner,sundance-grand-jury-priz...",2.0,5,Free-spirited fifteen-year-old Connie Wyatt ma...,[Free-spirited fifteen-year-old Connie Wyatt m...,9
2,90469,Paranormal Activity 3 (2011),"camerawork,boring,horror,ghosts,haunted-house,...",6.0,11,"In 1988, in California, cinematographer Dennis...","[In 1988, in California, cinematographer Denni...",4
3,267,Major Payne (1995),"comedy,military,children,damon-wayans,hilariou...",7.0,11,Major Benson Winifred Payne is being discharge...,[Major Benson Winifred Payne is being discharg...,5
4,3490,Horror Express (1972),"trains,bdr,dvdvideo,vhs",2.0,4,An English anthropologist has discovered a fro...,[An English anthropologist has discovered a fr...,5


In [12]:
docs_df.iloc[0]['sentences']

['It seems everyone is trying to get into heaven; at least those whose time is up.',
 "For Lance Barton, a struggling comedian and bicycle messenger, it's the last thing on his mind.",
 'His due date upstairs is 50 years away.',
 "In the meantime, he's got big dreams to pursue on Earth, such as landing a slot at the final Amateur Night Contest at the famed Apollo Theatre.",
 "Lance's has one little problem though - he ain't that funny.",
 "Thanks to an over-cautious emissary from heaven, Mr. Keyes, he's going to get hit (literally) with a much bigger problem.",
 'Showing that even God has difficulty finding good help these days, the inept minion mistakenly plucks Lance from a traffic accident - before it takes place.',
 'Transporting him to the Pearly Gates, or more accurately, the velvet roped-lines of the hottest club around, the error is finally addressed by Mr. King, the streetwise, no-nonsense head angel who manages the place from his plush windowed office.',
 'Since returning to 

In [13]:
docs_df.describe()

,movie_id,num_users,num_unique_tags,num_sentences
count,500.000000,500.000000,500.000000,500.000000
mean,31056.292000,15.528000,11.954000,9.962000
std,35468.482237,26.940542,8.402207,7.121938
min,10.000000,2.000000,2.000000,1.000000
25%,3493.750000,3.000000,4.000000,4.000000
50%,7145.500000,6.000000,9.000000,8.000000
75%,57558.750000,15.000000,20.000000,14.000000
max,128322.000000,227.000000,25.000000,38.000000


In [14]:
tok = TextTilingTokenizer(w=W, k=K)

In [15]:
def extract_segments(candidates):
    
    try:
        # we must manually insert "\n\n" because this is how 
        # texttilingtokenizer requires candidate boundaries to be 
        # represented.
        segments = tok.tokenize("\n\n".join(candidates))
    except ValueError:
        # this happens when the candidate list is too small for the 
        # text tiling tokenizer to be able to find segments. so just return
        # the original sentences.
        segments= candidates
        
    # now remove the artificially added chars
    segments = [segment.replace("\n\n"," ").strip() for segment in segments]
    
    return segments

In [16]:
%%time
docs_df['segments'] = docs_df['sentences'].map(lambda candidates: extract_segments(candidates))

CPU times: user 5.4 s, sys: 0 ns, total: 5.4 s
Wall time: 5.43 s


In [17]:
docs_df['segments'][0]

["It seems everyone is trying to get into heaven; at least those whose time is up. For Lance Barton, a struggling comedian and bicycle messenger, it's the last thing on his mind.",
 "His due date upstairs is 50 years away. In the meantime, he's got big dreams to pursue on Earth, such as landing a slot at the final Amateur Night Contest at the famed Apollo Theatre. Lance's has one little problem though - he ain't that funny. Thanks to an over-cautious emissary from heaven, Mr. Keyes, he's going to get hit (literally) with a much bigger problem. Showing that even God has difficulty finding good help these days, the inept minion mistakenly plucks Lance from a traffic accident - before it takes place.",
 "Transporting him to the Pearly Gates, or more accurately, the velvet roped-lines of the hottest club around, the error is finally addressed by Mr. King, the streetwise, no-nonsense head angel who manages the place from his plush windowed office. Since returning to his own body on Earth is

In [18]:
segments = docs_df['segments'].values
documents = docs_df['plot'].values
labelsets = truncate_labels(docs_df["unique_tags"].map(lambda tagstring: tagstring.split(",")).values,MIN_LABEL_DF)

In [19]:
def make_train_dataset(distance_matrix, medoid_indices):
    """
    Returns a matrix where element Aij contains the distance from sample i to medoid j.

    :param distance_matrix: MxM matrix with pairwise distances
    :param medoid_indices: array of length N containing the indices of the medoids for each cluster
    :return: distances to medoids (MxN matrix)
    """

    return distance_matrix[:,medoid_indices]

In [20]:
def make_test_dataset(source_vectorized_segments, medoid_vectorized_segments):
    """
    Calculates the distances from every source_document (reprsented by its segments) to every medoid
    document (also represented by its segments) using the hausdorff distance.
    
    Returns a matrix where element Aij contains the distance from sample i to medoid j.

    :param source_vectorized_segments: array of length M, where each element is a matrix with one row
        for every segment in a source document
    :param medoid_vectorized_segments: array of length N where each element is a matrix with one row
        for every segment in a medoid document
    :return: distances to medoids (MxN matrix)
    """
    
    num_test_samples = len(source_vectorized_segments)
    num_medoids = len(medoid_vectorized_segments)
    
    test_dataset = np.zeros((num_test_samples,num_medoids))    
    
    for i,source_segments in enumerate(source_vectorized_segments):
        for j,medoid_segments in enumerate(medoid_vectorized_segments):
            test_dataset[i][j] = hausdorff(source_segments.toarray(),medoid_segments.toarray())
            
    return np.array(test_dataset)

In [21]:
# segments, documents and labelsets are defined outside of the parameterGrid loop
# because they're the same for every configuration    
segments_train, segments_test, documents_train, documents_test, y_train, y_test = train_test_split(segments,
                                                                                               documents,
                                                                                               labelsets,
                                                                                               test_size=0.25)

print("number of documents in train set: {}".format(len(documents_train)))
print("number of documents in test set: {}".format(len(documents_test)))

# the binarizer needs to be fit on all labels
mlb = MultiLabelBinarizer()
mlb.fit(labelsets)

y_train = mlb.transform(y_train)
y_test = mlb.transform(y_test)

print("total number of unique tags: {} ".format(len(mlb.classes_)))

number of documents in train set: 375
number of documents in test set: 125
total number of unique tags: 199 


In [22]:
parameters = [{
    'stop_words': STOP_WORDS,
    'medoid_normalization':  MEDOID_NORMALIZATION,
    'svm_kernel': SVM_KERNEL,
    'svm_gamma': SVM_GAMMA,
    'svm_c':SVM_C,
    'svm_degree' :SVM_DEGREE,
    'norm': VECTORIZER_NORM
}]

In [23]:
print('total number of configurations to test: {}'.format(len(ParameterGrid(parameters))))

total number of configurations to test: 648


In [24]:
best_score = float("-inf")
best_configuration = None

for (i,configuration) in enumerate(tqdm(ParameterGrid(parameters))):
    
    # TFIDF_VECTORIZER = COUNT_VECTORIZER + TFIDF_TRANSFORMER
    tfidf_vectorizer = TfidfVectorizer(max_features=MAX_NB_WORDS, 
                                       preprocessor=PREPROC, 
                                       stop_words=configuration['stop_words'],
                                       norm = configuration['norm'])
    
    # TRAINING SET
    tfidf_vectorizer.fit(documents_train)
    tfidf_segments_train = vectorize_segments(segments_train, tfidf_vectorizer)
    
    
    # THE FOLLOWING BLOCK TAKES SOME TIME, BUT IT WILL ONLY RUN ONCE
    norm = configuration['norm']
    stop_words = configuration['stop_words']
    path_to_cache = INTERIM_DATA_ROOT.rstrip('/') + "/mimlsvm/distance-matrix-train-sample-{}-{}-{}.p".format(
        NB_DOCS,
        norm,
        stop_words)
    if os.path.isfile(path_to_cache):
        dist_matrix_train = pickle.load(open(path_to_cache,"rb"))
    else:
        print('Fitting distance matrix for norm={} and stop_words={}'.format(norm,stop_words))
        dist_matrix_train = make_distance_matrix_for_segments(tfidf_segments_train)
        pickle.dump(dist_matrix_train, open(path_to_cache, "wb"))
    
    # nb_medoids depends upon the dataset length
    nb_medoids = int(len(tfidf_segments_train) * SAMPLE_TO_NB_MEDOIDS_RATIO)
    
    medoids_indices_train = k_medoids(dist_matrix_train,nb_medoids)[0]

    X_train = make_train_dataset(dist_matrix_train,medoids_indices_train)
    
    # TEST SET
    
    # vectorizer has been fit on the training set only
    tfidf_segments_test = vectorize_segments(segments_test, tfidf_vectorizer)
    # medoids trained on the training set
    fitted_medoids = tfidf_segments_train[medoids_indices_train]
    X_test = make_test_dataset(tfidf_segments_test,fitted_medoids)
    
    # RUNNING
    if configuration['medoid_normalization'] == 'standard':      
        scaler = StandardScaler()
        X_train_final = scaler.fit_transform(X_train)
        X_test_final = scaler.transform(X_test)
    elif configuration['medoid_normalization'] == 'minmax':
        scaler = MinMaxScaler()
        X_train_final = scaler.fit_transform(X_train)
        X_test_final = scaler.transform(X_test)
    else:
        X_train_final = X_train
        X_test_final = X_test
    
    clf = OneVsRestClassifier(
        SVC(kernel=configuration['svm_kernel'],
            gamma=configuration['svm_gamma'],
            C=configuration['svm_c'],
            degree=configuration['svm_degree']),n_jobs=4)    
        
    # y_train was defined outside the loop    
    clf.fit(X_train_final,y_train)
    
    # train score
    y_train_preds = clf.predict(X_train_final)
    y_train_trues = y_train
    
    # validation score
    y_test_preds = clf.predict(X_test_final)
    y_test_trues = y_test
    
    train_score = f1_score(y_train_trues,y_train_preds, average='micro')
    val_score = f1_score(y_test_trues,y_test_preds,average='micro')
    
    print('iter: {}, train_score: {}, validation score: {}, configuration: {}'.format(i,
                                                                                      train_score,
                                                                                      val_score,
                                                                                      configuration))
    
    if val_score > best_score:
        best_score = val_score
        best_configuration = configuration
    

  0%|          | 0/648 [00:00<?, ?it/s]

Fitting distance matrix for norm=l1 and stop_words=None


  0%|          | 1/648 [00:20<3:42:05, 20.60s/it]

iter: 0, train_score: 0.09317166560306318, validation score: 0.0, configuration: {'medoid_normalization': 'standard', 'stop_words': None, 'svm_degree': 3, 'svm_gamma': 0.1, 'norm': 'l1', 'svm_c': 0.01, 'svm_kernel': 'poly'}


  0%|          | 2/648 [00:23<2:43:59, 15.23s/it]

...

iter: 645, train_score: 0.9996654399464704, validation score: 0.051207022677395755, configuration: {'medoid_normalization': None, 'stop_words': 'english', 'svm_degree': 5, 'svm_gamma': 0.2, 'norm': None, 'svm_c': 1, 'svm_kernel': 'poly'}


100%|█████████▉| 647/648 [39:59<00:05,  5.34s/it]

iter: 646, train_score: 1.0, validation score: 0.0361247947454844, configuration: {'medoid_normalization': None, 'stop_words': 'english', 'svm_degree': 5, 'svm_gamma': 0.5, 'norm': None, 'svm_c': 1, 'svm_kernel': 'poly'}


100%|██████████| 648/648 [40:04<00:00,  5.19s/it]

iter: 647, train_score: 1.0, validation score: 0.06000000000000001, configuration: {'medoid_normalization': None, 'stop_words': 'english', 'svm_degree': 5, 'svm_gamma': 1.0, 'norm': None, 'svm_c': 1, 'svm_kernel': 'poly'}


In [25]:
print('BEST SCORE: {}'.format(best_score))
print('BEST CONFIGURATION: {}'.format(best_configuration))

BEST SCORE: 0.07937877480586712
BEST CONFIGURATION: {'medoid_normalization': None, 'stop_words': 'english', 'svm_degree': 4, 'svm_gamma': 0.5, 'norm': None, 'svm_c': 1, 'svm_kernel': 'poly'}
